In [4]:
import numpy as np
import re

# Turner Rule Parameter File
with open("rna_turner2004.par") as file1:
  file_content = file1.read()
file1.close()
# Split the file content into sections
hairpin = file_content[file_content.find('# hairpin'):file_content.find('# hairpin_enthalpies')].split('\n',1)[1]
mismatch_interior = file_content[file_content.find('# mismatch_interior'):file_content.find('# mismatch_interior_enthalpies')].split('\n',1)[1]
bulge = file_content[file_content.find('# bulge'):file_content.find('# bulge_enthalpies')].split('\n',1)[1]
stack = file_content[file_content.find('# stack'):file_content.find('# stack_enthalpies')].split('\n',1)[1]
interior = file_content[file_content.find('# interior'):file_content.find('# interior_enthalpies')].split('\n',1)[1]
mismatch_hairpin = file_content[file_content.find('# mismatch_hairpin'):file_content.find('# mismatch_hairpin_enthalpies')].split('\n',1)[1]


def parse_data(data, pattern, has_header=False, header_length=0):
    energies = {}
    if has_header:
      base_pair_1 = data.split('\n')[0].split()[1:header_length+1]

    # Extract values for every pattern match
    for match in re.finditer(pattern, data):
        # Get header_length number of energy_values
        energy_values = tuple(map(int, match.groups()[:header_length]))
        if has_header: # next group has second base pair
          base_pair_2 = match.group(header_length+1)
        else:
          base_pair_1 = match.group(6)
          base_pair_2 = match.group(7)

        if has_header: # header has first base pair
          for i in range(header_length):
            energies[(base_pair_1[i], base_pair_2)] = energy_values[i]
        else:
          for i,v in enumerate(['E','A','C','G','U']):
            energies[(base_pair_1, base_pair_2+v)] = energy_values[i]

    return energies

stack_values = parse_data(stack, r'\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+/\*\s+([AUGCNS]+)\s*\*/', has_header=True, header_length=7)
mismatch_hairpin_values = parse_data(mismatch_hairpin, r'\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+/\*\s+([AUGCNS]+),([AUGCE])\s*\*/', has_header=False, header_length=5)
mismatch_interior_values = parse_data(mismatch_interior, r'\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+(-?\d+)\s+/\*\s+([AUGCNS]+),([AUGCE])\s*\*/', has_header=False, header_length=5)

hairpin_values = {}
for i, value in enumerate(hairpin.split()):
  hairpin_values[i] = int(value) if value != 'INF' else np.inf

bulge_values = {}
for i, value in enumerate(bulge.split()):
  bulge_values[i] = int(value) if value != 'INF' else np.inf

interior_values = {}
for i, value in enumerate(interior.split()):
  interior_values[i] = int(value) if value != 'INF' else np.inf

In [5]:
def FH(i, j, sequence):
    # i,j are indices of base pairs closing the loop

    loop_size = j - i - 1  # Number of unpaired bases

    if loop_size < 3:
        return float('inf')

    # Base pair closing the loop
    bp_closing = (sequence[i], sequence[j])

    # Loop size penalty
    loop_penalty = hairpin_values.get(loop_size, hairpin_values[max(hairpin_values.keys())])

    # Mismatch penalty
    if loop_size >= 4:
        mismatch_values = (bp_closing, sequence[i + 1] + sequence[j-1])
        mismatch_penalty = mismatch_hairpin_values.get(mismatch_values, 0)
    else:
        mismatch_penalty = 0

    return loop_penalty + mismatch_penalty

def FL(i, j, h, k, sequence):
    # i,j are outer base pair indices
    # h,k are inner base pair indices

    outer_bp = sequence[i] + sequence[j]
    inner_bp = sequence[h] + sequence[k]
    loop_size_1 = h - i - 1  # Unpaired bases on the left side
    loop_size_2 = j - k - 1  # Unpaired bases on the right side

    # Stack region (no unpaired bases)
    if loop_size_1 == 0 and loop_size_2 == 0:
        return 'S', stack_values.get((outer_bp, inner_bp), float('inf'))

    # Bulge loop (unpaired bases on one side)
    if loop_size_1 == 0 or loop_size_2 == 0:
        bulge_size = max(loop_size_1, loop_size_2)
        bulge_penalty = bulge_values.get(bulge_size, bulge_values[max(bulge_values.keys())])
        return 'B', bulge_penalty

    # Interior loop (unpaired bases on both sides)
    if loop_size_1 > 0 and loop_size_2 > 0:
        interior_size = loop_size_1 + loop_size_2
        interior_penalty = interior_values.get(interior_size, interior_values[max(interior_values.keys())])

        # Add mismatch penalties
        mismatch_penalty_1 = mismatch_interior_values.get((outer_bp, sequence[i + 1] + sequence[j-1]), 0)
        mismatch_penalty_2 = mismatch_interior_values.get((inner_bp, sequence[h - 1] + sequence[k+1]), 0)
        return 'I', interior_penalty + mismatch_penalty_1 + mismatch_penalty_2
    return 'N', float('inf')


In [19]:
def is_paired(i, j, sequence):
    pairs = {'A': 'U', 'U': 'A', 'G': 'C', 'C': 'G'}
    return pairs.get(sequence[i]) == sequence[j]

def compute_dp(sequence, MIN_LOOP_SIZE=4):
    n = len(sequence)

    # Initialize matrices
    V = np.full((n, n), float('inf')) # loops matrix
    W = np.full((n, n), float('inf')) # overall matrix
    P = np.full((n, n), -2) # note pairs/partitions
    l = np.full((n, n, 2), -1) # bounds for partitions


    for j in range(n):
      for i in range(n-1,-1,-1):
        if i >= j:
          continue
        if j - i + 1 < MIN_LOOP_SIZE:
          continue
        # Compute V[i, j]
        if is_paired(i, j, sequence):
          # hairpin (1st case)
          V[i, j] = FH(i, j, sequence)
          l[i, j, 0] = i + 1
          l[i, j, 1] = j - 1
          for h in range(i + 1, j):
            for k in range(h + 1, j):
              # 2nd order loop (2nd case)
                loop_type, second_order_energy = FL(i, j, h, k, sequence)

                if second_order_energy + V[h, k] < V[i, j] and (j - i + 1) >= MIN_LOOP_SIZE:
                  V[i, j] = second_order_energy + V[h, k]
                  if loop_type == 'S':
                    # Stack
                    l[i, j, 0] = i+1
                    l[i, j, 1] = j-1
                  elif loop_type == 'B' or loop_type == 'I':
                    # Bulge or Interior
                    l[i, j, 0] = h
                    l[i, j, 1] = k

          for k in range(i + 1, j):
            # bifurcation loop (3rd case)
            if W[i,k] + W[k+1,j] < V[i,j]:
              V[i,j] = W[i,k] + W[k+1,j]
              l[i,j,0] = k
              l[i,j,1] = k

        # Compute W[i, j]
        # use loop matrix (3rd case)
        W[i,j] = min(V[i,j], W[i,j])
        if is_paired(i, j, sequence):
          P[i, j] = -1
        else:
          P[i,j] = -2
        if i + 1 < n:
          # skip left (1st case)
          W[i, j] = min(W[i, j], W[i + 1, j])
        if j - 1 >= 0:
          # skip right (2nd case)
          W[i, j] = min(W[i, j], W[i,j-1])
        for k in range(i+1, j):
          # partition (4th case)
          if W[i,k] + W[k+1,j] < W[i,j]:
            W[i,j] = W[i,k] + W[k+1,j]
            P[i,j] = k

    return V, W, P, l

def traceback(i, j, sequence, P, l, MIN_LOOP_SIZE=4):
    structure = ['.'] * len(sequence)
    stack = [(i, j)]

    while stack:
      x, y = stack.pop()
      if x >= y:
        continue
      if y - x + 1 < MIN_LOOP_SIZE:
        continue
      k = P[x, y]  # Split point for current subsequence

      if k == -1:  # Hairpin or base-pair
        structure[x] = '('
        structure[y] = ')'
        ip = l[x, y, 0]  # Left boundary
        jp = l[x, y, 1]  # Right boundary
        if ip != jp:
          # Base pair between x,y
          stack.append((ip, jp))
        else:
          # Bifurcation: Split into parts
          stack.append((x + 1, ip))
          stack.append((ip + 1, y - 1))
      elif k >= 0:
        # Split sequence at k
        stack.append((x, k))
        stack.append((k + 1, y))
      elif x + 1 < y and W[x, y] == W[x + 1, y]:
        # Skip left
        stack.append((x + 1, y))
      elif x < y - 1 and W[x, y] == W[x, y - 1]:
        # Skip right
        stack.append((x, y - 1))

    return ''.join(structure)


In [7]:
with open('rnafold.small.rna_turner1999.par.mfe.gold', 'r') as testfile:
  data = testfile.read()
testfile.close()
data = data.split('\n')
for i in range(int(len(data)/2)):
  sequence = data[i*2]
  [result, mfe] = data[i*2+1].split(' ')
  mfe = float(mfe[1:len(mfe)-1])
  print(sequence)
  print('Original')
  print(result)
  print(mfe)
  V, W, P, l = compute_dp(sequence)
  mfe2 = W[0,len(sequence)-1]/100
  print('New')
  print(traceback(0, len(sequence)-1, sequence, P, l))
  print(mfe2)
  print('Diff')
  print(mfe-mfe2)


AGACGACAAGGUUGAAUCGCACCCACAGUCUAUGAGUCGGUGACAACAUUACGAAAGGCUGUAAAAUCAAUUAUUCACCACAGGGGGCCCCCGUGUCUAG
Original
((((.((..(((.((((.......(((((((.(((((.(....).)).)))....)))))))..........)))))))...(((....)))))))))..
-24.03
New
((((.(....((((.....((((......((...))..)))).))))..)(((.....((((..(((.(..))))....))))(((..))))))))))..
-21.2
Diff
-2.830000000000002
CCACUCAUUAAGUCCAAAGCUACGAGCGACAUUUCGAUGGGUGCGUCGACUGCGCUUACAGCGGCGGGGUGUAGACGACCUGUGUGAUAAUCUUUAAUAU
Original
.(((..............(((..(((((.((..((((((....)))))).)))))))..))).(((((.((....)).))))))))..............
-26.4
New
.(((.((....(((....(((...)))))).....(.......((((.((..((((...)))).......)).))))..))).)))..............
-24.4
Diff
-2.0
UUAGGCAGUCGCCGCUCGUUCGCGUGUAUGCUCCUUUGUCAGUCAGCCUACUAUAGCACUGCUCCAACUGGAUUCGGCAGCGUUCGCAUAGCGUCCUUGC
Original
...(((....)))((..(..((((((((((((((...(.((((..((........)))))))(((....)))...)).)))))..)))).)))..)..))
-25.9
New
..(.((....((((...(..(........))(((.(((.((((..((.((...))))))))...)))..)))..)))).))...(((.

In [17]:
from sklearn.metrics import f1_score, precision_score, recall_score

def calculate_metrics(predicted_structure, reference_structure):
    tp = sum(1 for i in range(len(predicted_structure)) if predicted_structure[i] == '(' and reference_structure[i] == '(')
    fp = sum(1 for i in range(len(predicted_structure)) if predicted_structure[i] == '(' and reference_structure[i] != '(')
    fn = sum(1 for i in range(len(reference_structure)) if reference_structure[i] == '(' and predicted_structure[i] != '(')

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    sensitivity = tp / (tp + fn) if tp + fn > 0 else 0
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if precision + sensitivity > 0 else 0

    return precision, sensitivity, f1
total_mfe_diff = 0
total_normalize_mfe_diff = 0
total_precision = 0
total_sensitivity = 0
total_f1 = 0

# Loop over each sequence
for i in range(int(len(data)/2)):
    sequence = data[i*2]
    reference_structure, mfe = data[i*2+1].split(' ')
    mfe = float(mfe[1:-1])

    V, W, P, l = compute_dp(sequence)
    predicted_structure = traceback(0, len(sequence)-1, sequence, P, l)
    predicted_mfe = W[0, len(sequence)-1] / 100

    precision, sensitivity, f1 = calculate_metrics(predicted_structure, reference_structure)
    total_mfe_diff += mfe - predicted_mfe
    total_normalize_mfe_diff += (mfe - predicted_mfe) / len(sequence)

    # Accumulate the metrics
    total_precision += precision
    total_sensitivity += sensitivity
    total_f1 += f1

    # Print results for each sequence
    print(f"Sequence: {sequence}")
    print(f"Original Structure: {reference_structure}")
    print(f"Predicted Structure: {predicted_structure}")
    print(f"Original MFE: {mfe}")
    print(f"Predicted MFE: {predicted_mfe}")
    print(f"Difference in MFE: {mfe - predicted_mfe}")
    print(f"Normalized Difference in MFE: {(mfe - predicted_mfe) / len(sequence)}")
    print(f"Precision: {precision:.3f}, Sensitivity: {sensitivity:.3f}, F1-Score: {f1:.3f}")

# Calculate and print the averages of the metrics
num_sequences = len(data) // 2
average_precision = total_precision / num_sequences
average_sensitivity = total_sensitivity / num_sequences
average_f1 = total_f1 / num_sequences

print(f"Average Precision: {average_precision:.3f}")
print(f"Average Sensitivity: {average_sensitivity:.3f}")
print(f"Average F1-Score: {average_f1:.3f}")
print(f"Average Difference in MFE: {total_mfe_diff / num_sequences}")
print(f"Average Normalized Difference in MFE: {total_normalize_mfe_diff / num_sequences}")


Sequence: AGACGACAAGGUUGAAUCGCACCCACAGUCUAUGAGUCGGUGACAACAUUACGAAAGGCUGUAAAAUCAAUUAUUCACCACAGGGGGCCCCCGUGUCUAG
Original Structure: ((((.((..(((.((((.......(((((((.(((((.(....).)).)))....)))))))..........)))))))...(((....)))))))))..
Predicted Structure: ((((.(....((((.....((((......((...))..)))).))))..)(((.....((((..(((.(..))))....))))(((..))))))))))..
Original MFE: -24.03
Predicted MFE: -21.2
Difference in MFE: -2.830000000000002
Normalized Difference in MFE: -0.02830000000000002
Precision: 0.414, Sensitivity: 0.414, F1-Score: 0.414
Sequence: CCACUCAUUAAGUCCAAAGCUACGAGCGACAUUUCGAUGGGUGCGUCGACUGCGCUUACAGCGGCGGGGUGUAGACGACCUGUGUGAUAAUCUUUAAUAU
Original Structure: .(((..............(((..(((((.((..((((((....)))))).)))))))..))).(((((.((....)).))))))))..............
Predicted Structure: .(((.((....(((....(((...)))))).....(.......((((.((..((((...)))).......)).))))..))).)))..............
Original MFE: -26.4
Predicted MFE: -24.4
Difference in MFE: -2.0
Normalized Difference in MFE: -0.02
Precis